In [1]:
import pandas as pd
from json_functions import create_df_simple, create_example

# players_list, df = create_df('test.json')

players_list, df = create_df_simple('very-big.json')

In [3]:
import numpy as np
import tensorflow as tf

In [4]:
features, outputs = df["rosters vector"], df['rating vector']

features = pd.DataFrame(features.values.tolist(), index= df.index)
outputs = pd.DataFrame(outputs.values.tolist(), index= df.index)

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a TensorFlow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, outputs), batch_size)

In [6]:
import tensorflow as tf

In [8]:
# Set up use GPU
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [9]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [18]:
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(100, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(50, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model

In [19]:
n_inputs, n_outputs = features.shape[1], outputs.shape[1]

In [20]:
model = get_model(n_inputs, n_outputs)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.33, random_state=42)

In [22]:
model.fit(X_train, y_train, verbose=1, epochs=100)
mae = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/100
184/184 [==============================] - 1s 3ms/step - loss: 0.4777
Epoch 2/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1930
Epoch 3/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1863
Epoch 4/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1793
Epoch 5/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1747
Epoch 6/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1683
Epoch 7/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1672
Epoch 8/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1646
Epoch 9/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1587
Epoch 10/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1589
Epoch 11/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1551
Epoch 12/100
184/184 [==============================] - 1s 3ms/step - loss: 0.1570
Epoch 13/100


In [17]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               106900    
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
Total params: 112,460
Trainable params: 112,460
Non-trainable params: 0
_________________________________________________________________


In [26]:
example = create_example(['EliGE', 'Stewie2K', 'Grim', 'FalleN', 'NAF'], ['device', 'Xyp9x', 'Magisk', 'dupreeh', 'gla1ve'], players_list)
model.predict(np.asarray([example]))

array([[1.0437515 , 1.0912896 , 0.7971262 , 0.7169643 , 0.61621076,
        1.6940743 , 1.4507071 , 1.166726  , 0.9371377 , 0.82347596]],
      dtype=float32)

In [42]:
def print_prediction(example, players_list, model):
    player_indices = [i for i, x in enumerate(example) if x == 1]
    example_formatted = np.asarray([example])
    pred = model.predict(example_formatted)
    print("Predicted rating vector:", pred[0])

    print("Team 1:")
    for player in player_indices[0:5]:
        print(players_list[player]+', ', end='')
    print("\nRating spread:")
    for i in range(0, 5):
        print(round(pred[0][i], 2))
    print("Team 2:")
    for player in player_indices[5:10]:
        print(players_list[player-len(players_list)]+', ', end='')
    print("\nRating spread:")
    for i in range(5, 10):
        print(round(pred[0][i], 2))



In [43]:
# This model unfortunately isn't able to predict rating by player, just the spread of rating for the team.
print_prediction(example, players_list, model)

Predicted rating vector: [1.0437515  1.0912896  0.7971262  0.7169643  0.61621076 1.6940743
 1.4507071  1.166726   0.9371377  0.82347596]
Team 1:
EliGE, NAF, Stewie2K, FalleN, Grim, 
Rating spread:
1.04
1.09
0.8
0.72
0.62
Team 2:
device, dupreeh, Xyp9x, gla1ve, Magisk, 
Rating spread:
1.69
1.45
1.17
0.94
0.82


Compare these results to a couple real matches played between these two teams on March 26th that the model hasn't seen:

https://www.hltv.org/stats/matches/mapstatsid/117239/liquid-vs-astralis?rankingFilter=Top10

https://www.hltv.org/stats/matches/mapstatsid/117231/astralis-vs-liquid?rankingFilter=Top10